In [ ]:
from __future__ import unicode_literals
import youtube_dl
import os
import pandas as pd

## Script for Youtube Download

In [ ]:
class MyLogger(object):
    def debug(self, msg):
        pass

    def warning(self, msg):
        pass

    def error(self, msg):
        print(msg)


def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

def download_clip(url, name, outdir):

    filename = os.path.join(outdir,name)

    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': filename,
        'noplaylist': True,
        'continue_dl': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192', }],
        'logger': MyLogger(),
        'progress_hooks': [my_hook],
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.cache.remove()
            info_dict = ydl.extract_info(url, download=False)
            ydl.prepare_filename(info_dict)
            ydl.download([url])
            return True
    except Exception:
        return False

### Donwload youtube songs

In [ ]:
filelist = "list.csv"
outdir = "/mnt/dev/dirceusilva/dados/Cover/SHS100k"


In [ ]:
list_songs_df = pd.read_csv(filelist, sep='\t', names=["set_id", "ver_id", "title", "performer", "url", "status"], header=None)


In [ ]:
index_failed=[]
for k in range(107205,len(list_songs_df)): #
    row = list_songs_df.iloc[k]
    print("{:d} de {:d} - {} - {}".format(k, len(list_songs_df), row["set_id"], row["ver_id"]))
    dirout = os.path.join(outdir, str(row["set_id"]))
    
    try:
        os.mkdir(dirout)
    except:
        print("Creation of the directory {} failed".format(row["set_id"]))
        
    name = str(row["ver_id"])+ '.%(ext)s'
    if not download_clip(row["url"], name, dirout):
        index_failed.append(k)

In [ ]:
print(len(index_failed))


In [4]:
import pandas as pd

shs100k = pd.read_csv("list.csv", sep='\t', header=None)

In [6]:
shs100k.columns=["set_id", "ver_id", "title", "performer", "url", "status"]

In [9]:
test = pd.read_csv("SHS100K-TEST", sep='\t', header=None)
test.columns = ["set_id", "ver_id"]
train = pd.read_csv("SHS100K-TRAIN", sep='\t', header=None)
train.columns = ["set_id", "ver_id"]
val = pd.read_csv("SHS100K-VAL", sep='\t', header=None)
val.columns = ["set_id", "ver_id"]


In [11]:
train["subset"] = "TRAIN"
test["subset"] = "TEST"
val["subset"] = "VAL"



In [13]:
shs100k = pd.merge(
    pd.merge(
        pd.merge(shs100k, train, on=["set_id", "ver_id"], how='left'), val, on=["set_id", "ver_id"], how='left'), 
         test, on=["set_id", "ver_id"], how='left')

In [21]:
import numpy as np

def get_subset(x):
    if x.subset_x not in [None, np.nan, "NaN"]:
        return x.subset_x
    elif x.subset_y not in [None, np.nan, "NaN"]:
        return x.subset_y
    elif x.subset not in [None, np.nan, "NaN"]:
        return x.subset
    

shs100k["subset"] = shs100k.apply(lambda x: get_subset(x), axis=1)

In [24]:
shs100k = shs100k.drop(["subset_x", "subset_y"], axis=1)

In [51]:
train["joined_id"] = train["set_id"].astype(str) + '_' + train["ver_id"].astype(str)
test["joined_id"] = test["set_id"].astype(str) + '_' + test["ver_id"].astype(str)
val["joined_id"] = val["set_id"].astype(str) + '_' + val["ver_id"].astype(str)


def get_subset(x):
    
    joined_id = str(x.set_id) + '_' + str(x.ver_id)
    if joined_id in train["joined_id"].values:
        return "TRAIN"
    elif joined_id in test["joined_id"].values:
        return "TEST"
    elif joined_id in val["joined_id"].values:
        return "VAL"
    

In [52]:
shs100k["subset"] = shs100k.apply(lambda x: get_subset(x), axis=1)

In [58]:
shs100k.to_csv("shs100k.csv", sep=';')

In [ ]:
pd.read_csv

In [59]:
shs100k

,set_id,ver_id,title,performer,url,status,subset
0,0,0,Summertime,Abbie Mitchell,https://www.youtube.com/watch?v=TMHWEcSKEPI,True,TRAIN
1,0,1,Summertime,Helen Jepson,https://www.youtube.com/watch?v=E3lw8gaHs04,True,TRAIN
2,0,2,Summertime,Billie Holiday and Her Orchestra,https://www.youtube.com/watch?v=aPpJPTJc1lU,True,TRAIN
3,0,3,Summertime,Jerry Kruger and Her Orchestra,https://www.youtube.com/watch?v=J9JctRqcDso,True,TRAIN
4,0,4,Overture and Summertime,Anne Brown,https://www.youtube.com/watch?v=R44waInkjgI,True,TRAIN
...,...,...,...,...,...,...,...
116348,9994,1,Sytten ti,Otto Brandenburg,https://www.youtube.com/watch?v=EVsRN0eDGNc,True,VAL
116349,9996,0,We Want the Airwaves,Ramones,https://www.youtube.com/watch?v=VJ5X721ABs4,True,VAL
116350,9998,0,I Don't Care,Ramones,https://www.youtube.com/watch?v=QveJv7V4OZI,True,VAL
116351,9998,1,I Don't Care,The Flatmates,https://www.youtube.com/watch?v=6L8jlQfp3wA,True,VAL


## Convert  Wav to MP3

In [ ]:
import glob

direct = '/mnt/dev/dirceusilva/dados/Cover/SHS100k'
#direct = '/home/dirceusilva/Documents/Test'

all_files = glob.glob(os.path.join(direct,'**/*.wav'))

for k,f in enumerate(all_files):
    print("{:d} - {:d} - {} ".format(k,len(all_files), f))
    
    fileout = f[:-3]+ "mp3"

    !lame --silent {f} {fileout}
    if os.path.exists(fileout):
        #print("converted")
        os.remove(f)
    



## List Failed  Downloads

In [ ]:
filelist = "list.csv"
outdir = '/mnt/dataset/public/covers/SHS100k'


In [ ]:
list_songs_df = pd.read_csv(filelist, sep='\t', names=["set_id", "ver_id", "title", "performer", "url", "status"], header=None)


In [ ]:
index_failed=[]

for k in range(len(list_songs_df)):
    row = list_songs_df.iloc[k]
    print("{:d} de {:d} - {} - {}".format(k, len(list_songs_df), row["set_id"], row["ver_id"]))
    dirout = os.path.join(outdir, str(row["set_id"])) 

    filepath = os.path.join(dirout,str(row["ver_id"])+ '.mp3')
    
    if not os.path.exists(filepath):
        try:
            os.mkdir(dirout)
        except:
            print("Creation of the directory {} failed".format(row["set_id"]))
            
        name = str(row["ver_id"])+ '.%(ext)s'
        if not download_clip(row["url"], name, dirout):
            index_failed.append((k, row["url"]))


### Save List of Failed Downloads

In [ ]:
index = []
for k, url_y in index_failed:
    index.append(k)

In [ ]:
df_1 = list_songs_df.iloc[index]

In [ ]:
df_1.to_csv("list_failed.csv", sep=";", header=False)